In [490]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

Попробуем предсказать оценку Роберта Паркера на основании погодных данных. 

In [491]:
file = 'df_month.xlsx'
xl = pd.ExcelFile(file)
df = xl.parse('Sheet1')
df.head()

,date,max,min,average,wind,rain,effect_t,year,month,margaux,pauillac,graves,pomerol,emilion
0,1975-01-01,15.433333,6.290323,10.877419,4.225806,2.967742,7.993548,1975,1,78,89,89,94,85
1,1975-02-01,16.285714,4.178571,9.985714,2.714286,2.214286,7.703571,1975,2,78,89,89,94,85
2,1975-03-01,13.419355,3.419355,8.354839,3.354839,2.064516,5.529032,1975,3,78,89,89,94,85
3,1975-04-01,18.800000,7.700000,12.753333,3.366667,1.800000,10.533333,1975,4,78,89,89,94,85
4,1975-05-01,18.466667,7.966667,13.446667,2.766667,1.633333,11.880000,1975,5,78,89,89,94,85


In [492]:
col = df.columns
col

Index(['date', 'max', 'min', 'average', 'wind', 'rain', 'effect_t', 'year',
       'month', 'margaux', 'pauillac', 'graves', 'pomerol', 'emilion'],
      dtype='object')

In [493]:
df.isnull().count()

date        504
max         504
min         504
average     504
wind        504
rain        504
effect_t    504
year        504
month       504
margaux     504
pauillac    504
graves      504
pomerol     504
emilion     504
dtype: int64

In [494]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504 entries, 0 to 503
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      504 non-null    datetime64[ns]
 1   max       504 non-null    float64       
 2   min       504 non-null    float64       
 3   average   504 non-null    float64       
 4   wind      504 non-null    float64       
 5   rain      504 non-null    float64       
 6   effect_t  504 non-null    float64       
 7   year      504 non-null    int64         
 8   month     504 non-null    int64         
 9   margaux   504 non-null    int64         
 10  pauillac  504 non-null    int64         
 11  graves    504 non-null    int64         
 12  pomerol   504 non-null    int64         
 13  emilion   504 non-null    int64         
dtypes: datetime64[ns](1), float64(6), int64(7)
memory usage: 55.2 KB


In [495]:
df1 = df.query('month == 1')
train = df1[['max', 'min', 'average', 'wind', 'rain', 'effect_t']]
target = df1['margaux']
train, test, y_train, y_test = train_test_split(train, target, test_size = 0.2, shuffle = False)

Создадим результирующий датафрейм, запишем в него тестовые данные

In [505]:
result1 = []
result1 = pd.DataFrame(y_test)
result1.reset_index(inplace = True)
result1.drop('index', inplace = True, axis = 1)
result1.head(10)

,margaux
0,97
1,95
2,87
3,89
4,80
5,90
6,94
7,97
8,90


В цикле сделаем помесячную выборку (месяцы возьмем с марта по август), обучим линейную регрессию и сделаем помесячные прогнозы.

In [506]:
for i in range(3, 9):
    df1 = df.query('month == @i')
    train = df1[['max', 'min', 'average', 'wind', 'rain', 'effect_t']]
    target = df1['margaux']
    train, test, y_train, y_test = train_test_split(train, target, test_size = 0.2, shuffle = False)
    lr = LinearRegression(normalize = True, n_jobs = -1)
    lr.fit(train, y_train)
    pred = lr.predict(test)
    pred1 = pd.DataFrame(pred, columns = [f'{i}'])
    result1 = result1.merge(pred1, right_index = True, left_index = True)

In [513]:
result1.head(10)

,margaux,3,4,5,6,7,8
0,97,91.820294,86.885725,90.786755,85.942981,87.060567,87.435134
1,95,88.773761,81.489605,90.190272,81.057901,87.788283,89.376291
2,87,88.913487,83.096417,86.554923,89.295642,84.547648,86.394409
3,89,91.494114,96.713596,86.607195,84.588430,81.967058,92.024768
4,80,85.936937,79.059698,83.366390,81.163258,93.820307,92.735744
5,90,81.560761,69.406390,59.799203,82.404249,68.043397,63.450161
6,94,79.885957,77.580256,81.698008,85.641745,82.609193,67.950775
7,97,85.486304,78.617888,69.268296,78.358292,82.639706,85.017353
8,90,83.698953,74.998258,77.530061,79.793288,88.661971,77.747443


Получившуюся таблицу снова поделим на обучающую и тестовую выборки. Проведем обучение и сделаем предсказание.

In [514]:
result1.columns

Index(['margaux', '3', '4', '5', '6', '7', '8'], dtype='object')

In [515]:
train = result1[['3', '4', '5', '6', '7', '8']]
target = result1['margaux']
train, test, y_train, y_test = train_test_split(train, target, test_size = 0.5, shuffle = False)
lr = LinearRegression()
lr.fit(train, y_train)
full = lr.predict(test)

In [516]:
full

array([88.53297954, 43.25372575, 76.16413116, 66.19442061, 78.71484043])

In [517]:
y_test

4    80
5    90
6    94
7    97
8    90
Name: margaux, dtype: int64

Рассчитаем метрику качества

In [518]:
mean_squared_error(y_test, full)

730.496532169647

Кошмар, с такими предсказаниями можно по миру пойти.

Решим ту же самую задачу с помощью случайного леса

In [521]:
df1 = df.query('month == 1')
train = df1[['max', 'min', 'average', 'wind', 'rain', 'effect_t']]
target = df1['margaux']
train, test, y_train, y_test = train_test_split(train, target, test_size = 0.2, shuffle = False)

In [522]:
result2 = []
result2 = pd.DataFrame(y_test)
result2.reset_index(inplace = True)
result2.drop('index', inplace = True, axis = 1)
result2.head(10)

,margaux
0,97
1,95
2,87
3,89
4,80
5,90
6,94
7,97
8,90


In [523]:
from sklearn.ensemble import RandomForestRegressor
for i in range(3, 9):
    df1 = df.query('month == @i')
    train = df1[['max', 'min', 'average', 'wind', 'rain', 'effect_t']]
    target = df1['margaux']
    train, test, y_train, y_test = train_test_split(train, target, test_size = 0.2, shuffle = False)
    forest = RandomForestRegressor(max_depth = 11, n_estimators = 5, n_jobs = -1)
    forest.fit(train, y_train)
    pred = forest.predict(test)
    pred1 = pd.DataFrame(pred, columns = [f'{i}'])
    result2 = result2.merge(pred1, right_index = True, left_index = True)

In [525]:
result2.head(10)

,margaux,3,4,5,6,7,8
0,97,83.4,92.6,86.4,88.0,80.4,83.2
1,95,87.4,83.4,86.4,85.0,86.6,88.6
2,87,90.6,87.8,86.4,88.2,77.2,82.2
3,89,92.0,88.8,89.2,83.8,78.0,84.6
4,80,80.6,78.0,80.6,79.8,88.2,86.2
5,90,83.6,83.8,81.0,89.2,79.8,83.6
6,94,81.2,75.2,84.6,84.0,86.2,85.0
7,97,84.6,82.6,90.8,78.8,84.6,79.0
8,90,85.8,80.6,78.6,85.6,89.4,83.6


In [ ]:
train = result2[['3', '4', '5', '6', '7', '8']]
target = result2['margaux']
train, test, y_train, y_test = train_test_split(train, target, test_size = 0.5, shuffle = False)
forest = RandomForestRegressor()
forest.fit(train, y_train)
full = forest.predict(test)

In [528]:
full

array([93.6 , 93.76, 93.66, 92.9 , 94.26])

In [529]:
y_test

4    80
5    90
6    94
7    97
8    90
Name: margaux, dtype: int64

In [530]:
mean_squared_error(y_test, full)

46.834159999999976

Ну вот, вроде получше.

Теперь решим с помощью дерева решений

In [532]:
df1 = df.query('month == 1')
train = df1[['max', 'min', 'average', 'wind', 'rain', 'effect_t']]
target = df1['margaux']
train, test, y_train, y_test = train_test_split(train, target, test_size = 0.2, shuffle = False)

In [533]:
result3 = []
result3 = pd.DataFrame(y_test)
result3.reset_index(inplace = True)
result3.drop('index', inplace = True, axis = 1)
result3.head(10)

,margaux
0,97
1,95
2,87
3,89
4,80
5,90
6,94
7,97
8,90


In [534]:
from sklearn.tree import DecisionTreeRegressor
for i in range(3, 9):
    df1 = df.query('month == @i')
    train = df1[['max', 'min', 'average', 'wind', 'rain', 'effect_t']]
    target = df1['margaux']
    train, test, y_train, y_test = train_test_split(train, target, test_size = 0.2, shuffle = False)
    tree = DecisionTreeRegressor()
    tree.fit(train, y_train)
    pred = tree.predict(test)
    pred1 = pd.DataFrame(pred, columns = [f'{i}'])
    result3 = result3.merge(pred1, right_index = True, left_index = True)

In [536]:
result3.head(10)

,margaux,3,4,5,6,7,8
0,97,86.0,95.0,86.0,85.0,86.0,82.0
1,95,88.0,76.0,74.0,90.0,88.0,86.0
2,87,85.0,76.0,75.0,88.0,88.0,75.0
3,89,98.0,86.0,88.0,90.0,77.0,75.0
4,80,86.0,76.0,79.0,86.0,75.0,86.0
5,90,86.0,86.0,77.0,89.0,77.0,78.0
6,94,82.0,88.0,74.0,78.0,86.0,78.0
7,97,86.0,86.0,95.0,76.0,77.0,75.0
8,90,77.0,86.0,77.0,88.0,90.0,76.0


In [537]:
train = result3[['3', '4', '5', '6', '7', '8']]
target = result3['margaux']
train, test, y_train, y_test = train_test_split(train, target, test_size = 0.5, shuffle = False)
tree = DecisionTreeRegressor()
tree.fit(train, y_train)
full = tree.predict(test)

In [538]:
mean_squared_error(y_test, full)

76.0

In [539]:
full

array([97., 89., 89., 89., 89.])

In [540]:
y_test

4    80
5    90
6    94
7    97
8    90
Name: margaux, dtype: int64

Дерево решений показало результат хуже случайного леса, но лучше линейной регрессии